In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import load_counties_data

(counties_geojson, co_data_frame) = load_counties_data()


In [ ]:
from sportwarz import add_teams_and_distances

leagues = {
    'NFL': { "weight": 0.9 },
    'NBA': { "weight": 0.7 },
    'MLB': { "weight": 0.6 },
    'NHL': { "weight": 0.25 }, 
    'MLS': { "weight": 0.15 }
}    

add_teams_and_distances(leagues, co_data_frame)   

In [ ]:
from sportwarz import compute_shares
        
compute_shares(leagues, co_data_frame)  

In [ ]:
from sportwarz import compute_output_dataframes

compute_output_dataframes(leagues, co_data_frame)       

In [ ]:
from sportwarz import reset_county_styles, heatmap_counties

nhl_color_map = { 
    "St. Louis Blues": "blue",
    "Chicago Blackhawks": "black",
    "Minnesota Wild": "darkgreen",
    "Nashville Predators": "gold",
    "Detroit Red Wings": "red",
    "Dallas Stars": "limegreen",
    "Colorado Avalanche": "maroon",
    "Columbus Blue Jackets": "midnightblue",
    "Pittsburgh Penguins": "gold",
    "Philadelphia Flyers": "darkorange",
    "Washington Capitals": "navy",
    "New York Rangers": "red",
    "New York Islanders": "mediumblue",
    "New Jersey Devils": "black",
    "Buffalo Sabres": "navy",
    "Boston Bruins": "gold",
    "San Jose Sharks": "teal",
    "Utah Mammoth": "skyblue",
    "Los Angeles Kings": "black",
    "Anaheim Ducks": "orangered",
    "Seattle Kraken": "aquamarine",
    "Vegas Golden Knights": "goldenrod",
    "Tampa Bay Lightning": "navy",
    "Florida Panthers": "crimson",
    "Carolina Hurricanes": "red"
}

mlb_color_map = {
    "St. Louis Cardinals": "red",
    "Chicago Cubs": "blue",
    "Chicago White Sox": "black",
    "Kansas City Royals": "skyblue",
    "New York Yankees": "black",
    "Boston Red Sox": "firebrick",
    "Athletics": "yellow"
}

reset_county_styles(counties_geojson)
# heatmap_counties(leagues, counties_geojson, "MLB", mlb_color_map)
# heatmap_counties(leagues, counties_geojson, "MLB", mlb_color_map)     


In [ ]:
from sportwarz import render_map

render_map(leagues, counties_geojson, co_data_frame)

In [ ]:
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.0f' % x)
for league in leagues.keys():
    league_dfs = leagues[league]["dataframes"]
    league_dfs["share_of_pop"] = league_dfs['share'] * league_dfs["population"]
    group_cols = list(league_dfs.columns.difference(['team_name', 'share_of_pop']))
    print(league_dfs[['team_name', 'share_of_pop']].groupby('team_name').sum().sort_values(by='share_of_pop',ascending=False)) #.sort_values(by=['team_name'])